# Choropleth map Netherlands' municipalities using Python's Bokeh with GEOJSON and CBS data
  
Plotting a map of the Netherlands' municipalities using Pyton's Bokeh with GEOJSON was copied from https://github.com/Lo-o/Choropleth-map-Netherlands-Bokeh-Python. Where the original examples uses CBS data to fill the map, I use data from Vektis Open data. The data is prepared in another Notebook (1.Vektis Open data demography) within this folder. The export of this preparatory Notebook can be saved and then imported in this one.

## Initialize Notebook

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
from bokeh.io import output_notebook
from bokeh.plotting import figure, show


from bokeh.io import show, output_notebook, export_png
from bokeh.models import (
    GeoJSONDataSource,
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    ColorBar,
    LogTicker
)
from bokeh.palettes import Viridis256 as palette
from bokeh.plotting import figure

import json
from collections import OrderedDict
output_notebook()

# Select analysis

In [ ]:
# Do you want to load the information about outliers or information about the whole country?
analyse = 'outliers' #'all'

analyse_jaar = '2011'

## Get the data from Vektis dump
We load the data directly into a pandas dataframe

In [ ]:
if analyse == 'outliers':
    df = pd.read_csv('data/outliers_' + analyse_jaar + '.csv', decimal=',')
    displayValue = 'Outliers ' + df.columns[1]
elif analyse ==  'all':
    df = pd.read_csv('data/df_vektis_' + analyse_jaar + '.csv', decimal=',')
    displayValue = df.columns[1] + ' per gemeente'
else:
    print('Please, use "outliers" or "all"')

# It's easier just to have a 'Value' column for further programming ;-)
df.columns = ['Gemeentenaam','Value']

# Take a look at the data from Vektis:
df.head()

The formatting of the names of the municipalities within the Vektis Open data is not the same as in the GEOJSON data. We'll need to fix this (Part I).
The names of the municipalities within the Vektis Open data and the GEOJSON data does not always match. We'll need to fix this (Part II).

In [ ]:
# dictionary contains (1) value from GEOJSON map and (2) Gemeentenaam from Vektis
conversion_dict = {'Dantumadeel (Dantumadiel)':'Dantumadiel',
                   'Ferwerderadeel (Ferwerderadiel)':'Ferwerderadiel',
                   'SÃºdwest-FryslÃ¢n':'Sudwest-Fryslan',
                   'Tietjerksteradeel (Tytsjerksteradiel)':'Tytsjerksteradiel',
                   'De Friese Meren':'De Fryske Marren',
                   'Franekeradeel':'Waadhoeke',
                   'het Bildt':'Waadhoeke',
                   'Littenseradeel (Littenseradiel)':'Waadhoeke',
                   'Menaldumadeel (Menaldumadiel)':'Waadhoeke',
                   'Leeuwarderadeel':'Leeuwarden',
                   'Tietjerksteradeel (Tytsjerksteradiel)':'Tytsjerksteradiel',
                   'Groesbeek':'Berg en Dal',
                   'Kollumerland en Nieuwkruisland':'Kollumerland Ca',
                   'Rijnwaarden':'Zevenaar',
                   'Bellingwedde':'Westerwolde',
                   'Vlagtwedde':'Westerwolde',
                   'Groningen (Gr)':'Groningen',
                   'Hoogezand-Sappemeer':'Midden-Groningen',
                   'Menterwolde':'Midden-Groningen',
                   'Slochteren':'Midden-Groningen',
                   'Bergen (L)':'Bergen Lb',
                   'Bergen (NH)':'Bergen Nh',
                   'Nuenen, Gerwen en Nederwetten':'Nuenen Ca',
                   'Schijndel':'Meierijstad',
                   'Sint-Oedenrode':'Meierijstad',
                   'Veghel':'Meierijstad',
                   "'s-Hertogenbosch":'S Hertogenbosch',
                   'Bussum':'Gooise Meren',
                   'Muiden':'Gooise Meren',
                   'Naarden':'Gooise Meren',
                   'Haarlemmerliede en Spaarnwoude':'Haarlemmerliede Ca',
                   'Zeevang':'Edam-Volendam',
                   'Hengelo (O)':'Hengelo',
                   'Utrecht (Ut)':'Utrecht',
                   "'s-Gravenhage":'S Gravenhage'}

def fixMunicipalities(df, convert_dict, column):
    """Fix the municipalities names so they match the GEOJSON data by (1) correcting the case, and (2) matching based
    on the dictionary which translates municipalities from Vektis to GEOJSON"""
    # Part I
    df['Gemeentenaam'] = df['Gemeentenaam'].str.title()
    df['Gemeentenaam'] = df['Gemeentenaam'].str.replace(' En ', ' en ')
    df['Gemeentenaam'] = df['Gemeentenaam'].str.replace(' Aan ', ' aan ')
    df['Gemeentenaam'] = df['Gemeentenaam'].str.replace(' De ', ' de ')
    df['Gemeentenaam'] = df['Gemeentenaam'].str.replace(' Den ', ' den ')
    df['Gemeentenaam'] = df['Gemeentenaam'].str.replace(' Op ', ' op ')
    df['Gemeentenaam'] = df['Gemeentenaam'].str.replace(' Van ', ' van ')
    df['Gemeentenaam'] = df['Gemeentenaam'].str.replace(' Bij ', ' bij ')
    df['Gemeentenaam'] = df['Gemeentenaam'].str.replace('Ij', 'IJ')
    
    df = df.set_index('Gemeentenaam')

    noMatch = []
    # Part II
    for key in convert_dict:
        try: 
            df[column][key] = df[column][convert_dict[key]]

        except KeyError:
            noMatch.append(convert_dict[key])
    
    return df, noMatch

df_fixed, unfindable = fixMunicipalities(df, conversion_dict, 'Value')
df_fixed.head()

The following municipalities were not matched. This happens a lot with the Outliers-files, but should be empty for Vektis files containing all municipalities.

In [ ]:
unfindable

(Almost) one-on-one copy of the original file from Github.

In [ ]:
# read all municipalities
# SOURCE: https://www.webuildinternet.com/2015/07/09/geojson-data-of-the-netherlands/

# os.chdir(r'LOCATION\OF\YOUR\FILE')

with open(r'Gemeenten.geojson', 'r') as f:
    dutch_municipalities_dict = json.loads(f.read(), object_hook=OrderedDict)
    
# See how this geojson looks. It's basically a dictionary with many polygons, 
# which dictate the outer edges of (in this case) the municipalities. 
# Also, there is an 'properties' part connected to each set of locations, which holds additional information. 
# Already included are the municipality names, code and some more features. 
# We will add one more feature ourselves here: the average regional income. 

# Can't run this in Github since it displays the entire dictionary which is very large. 
# dutch_municipalities_dict

In [ ]:
# Function below will add in the Vektis data we have. 
# This is based on crossing the names: it will look for the name from the geojson in the Vektis data
# This will work for many municipalities, but there will unfortunately be cases in which the municipality can't be found. 

# Python will give a KeyError in those cases. Because of changes in municipalities (Gemeentelijke herindelingen) 
# we won't be able to fix all name incompatabilities, but many we will. Therefore, store them in  a separate file.
def merge_values(dutch_municipalities_dict, df):
    
    municipality = dutch_municipalities_dict['properties']['name']
    
    try: 
        dutch_municipalities_dict['properties']['Value'] = round(df['Value'][municipality],0).astype('float')
    except KeyError:
        #unfindable.append(municipality)
        dutch_municipalities_dict['properties']['Value'] = df['Value'].mean()

    return dutch_municipalities_dict


#unfindable = []

# merge Vektis data: execute the function here.
dutch_municipalities_dict['features'] = [merge_values(municipality, df_fixed) for municipality in dutch_municipalities_dict['features']]

# See how the GEOJSON has changed since the previous one: a new property - Value - is added
#dutch_municipalities_dict

In [ ]:
unfindable

# Visualization in Bokeh

The original version at Github was modified for the use of the Vektis Open data.

In [ ]:
def plotMap(dict, minValue, maxValue, displayValue, jaar):
    color_mapper = LinearColorMapper(palette=palette, low_color='blue', high_color='red', low=minValue, high=maxValue)

    TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

    p = figure(
        title= displayValue + " (" + jaar + ")", tools=TOOLS,
        x_axis_location=None, y_axis_location=None
    )
    p.grid.grid_line_color = None

    geo_source = GeoJSONDataSource(geojson=json.dumps(dict))
    
    p.patches('xs', 'ys', source=geo_source,
              fill_color={'field': 'Value', 'transform': color_mapper},
              fill_alpha=0.7, line_color="white", line_width=0.5)

    hover = p.select_one(HoverTool)
    hover.point_policy = "follow_mouse"
    hover.tooltips = [
        ("Name", "@name"),
        #("Code", "@code"),
        ("Waarde", "@Value"),
        #("Level", "@level"),
        #("(Long, Lat)", "($x, $y)"),
    ]

    color_bar = ColorBar(color_mapper=color_mapper, border_line_color=None, location=(0,0))
    p.add_layout(color_bar, 'left')
    
    show(p)

minValue = int(math.floor(df_fixed.min() / 10.0)) * 10
maxValue = int(math.ceil( df_fixed.max() / 10.0)) * 10
plotMap(dutch_municipalities_dict, minValue, maxValue, displayValue, analyse_jaar)

# Make it possible to plot multiple maps using an all-in-one function

In [ ]:
def allInOne(jaar, analyse):
    brk = False
    if analyse == 'outliers':
        df = pd.read_csv('data/outliers_' + jaar + '.csv', decimal=',')
        displayValue = 'Outliers ' + df.columns[1]
    elif analyse == 'all':
        df = pd.read_csv('data/df_vektis_' + jaar + '.csv', decimal=',')
        displayValue = df.columns[1] + ' per gemeente'
    else:
        print('Please, select "outliers" or "all"')
        return
    
    df.columns = ['Gemeentenaam','Value']
    df_fixed, unfindable = fixMunicipalities(df, conversion_dict, 'Value')

    with open(r'Gemeenten.geojson', 'r') as f:
        dutch_municipalities_dict = json.loads(f.read(), object_hook=OrderedDict)
        
    dutch_municipalities_dict['features'] = [merge_values(municipality, df_fixed) for municipality in dutch_municipalities_dict['features']]
    
    minValue = int(math.floor(df_fixed.min() / 10.0)) * 10
    maxValue = int(math.ceil( df_fixed.max() / 10.0)) * 10
    
    plotMap(dutch_municipalities_dict, minValue, maxValue, displayValue, jaar)

allInOne('2012', 'all')
allInOne('2012', 'outliers')